In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                 dcc.Dropdown(id='site-dropdown',
                                 options=[            
                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                         {'label': 'All', 'value': 'All'}
                                         ],
                                     value='All',
                                     placeholder="Select Launch site here",
                                     searchable=True),
                                     html.Br(),
                                

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,value=(min_payload, max_payload)),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),Input(component_id='site-dropdown', component_property='value'))
# Add computation to callback function and return graph
def get_graph(dropdown):
    if dropdown=='All':
        fig = px.pie(spacex_df, names='Launch Site', values='class')
        return fig
    else:
        filtername=dropdown
        df=spacex_df[spacex_df['Launch Site']==dropdown]
        df1=df.groupby(['Launch Site','class']).size().reset_index()
        df1 = df1.rename(columns={0:'count'})
        fig = px.pie(df1, names='class', values='count')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),[Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def scatter_plot(dropdown,payload_range):
        if dropdown=='All':
            low, high = payload_range 
            mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
            scat = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class',color="Booster Version Category")
            return scat
        else:
            low, high = payload_range 
            df2=spacex_df[spacex_df['Launch Site']==dropdown]
            mask = (df2['Payload Mass (kg)'] > low) & (df2['Payload Mass (kg)'] < high)
            scat = px.scatter(df2[mask], x='Payload Mass (kg)', y="class", color="Booster Version Category")
            return scat

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET /_dash-component-suites/dash_core_components/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:26] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 10:06:27] "POST /_dash-update-component HTTP/1.1" 200 -
